In [9]:
text="""MODULE 06 Visible Surface Detection and Animation Visible Surface

 Visible surface detection or hidden surface removal When we view a picture containing non
transparent objects and surfaces, then we cannot see those objects from view which are behind from objects closer to eye
 We must remove these hidden surfaces to get a realistic screen image
 The identification and removal of these surfaces is called Hidden
surface problem
 There are two approaches for removing hidden surface problems − 1] Object
Space method 2] Image
space method The Object
space method is implemented in physical coordinate system and image
space method is implemented in screen coordinate system
 Back face Detection Method
 A fast and simple object
space method in which objects and parts of object are compared to find out the visible surfaces
 Algorithm
 Given in class Depth Buffer or Z−Buffer Method
 The drawback of Back face detection is that
 1) It cannot work on partial hidden surfaces 2) It works only with non
overlapping objects So, to eliminate these issues, we have depth buffer method
 It is an image
space approach
 The basic idea is to test the Z
depth of each surface to determine the closest visible surface
 In this method each surface is processed separately one pixel position at a time across the surface
 The depth values for a pixel are compared and the closest smallest z surface determines the color to be displayed in the frame buffer
 ( z value means distance between surface and view plane ) It is applied very efficiently on surfaces of polygon
 Surfaces can be processed in any order
 To override the closer polygons from the far ones, two buffers are used
 ● frame buffer / refresh buffer
 for intensity ● depth buffer
 for z values Algorithm and cases
 Given in class Advantages
 ● It is easy to implement
 ● It reduces the speed problem if implemented in hardware
 ● It processes one object at a time
 Area
Subdivision Method
 Notes given in class For detail reference of above topics
 https
//www
tutorialspoint
com/computer_graphics/visible_surface_detection
htm Animation
 Computer animation is a visual digital display technology that processes the moving images on screen
 In simple words, it can be put or defined as the art or power of giving life, energy and emotions etc
 to any non
living or inanimate object via computers
 It can be presented in the form of any video or movie
 Computer animation has the ability to make any dead image alive
 The key/main concept behind computer animation is to play the defined images at a faster rate to fool the viewer so that the viewer should interpret those images as a continuous motion of images
 Principles of traditional animation
 Notes given in class Animation Techniques
 1] Frame by frame 2] Procedural 3] Behavioral 4] Key framing 5] Motion capture 6] DynamicsKey Framing
 IMP Key framing is a technique in computer graphics that uses keyframes to define the starting and ending points of an object or character's movement or pose Diagram and notes
 Given in class
"""

In [11]:
from transformers import pipeline
#from Read_file import process_file
from sentence_transformers import SentenceTransformer, util
import json
import time
import random
from datetime import datetime

In [12]:
class QuestionSystem:
    def __init__(self):
        self.question_generator = pipeline("text2text-generation", model="google/flan-t5-large")
        self.similarity_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
        self.qa_pairs = []
        self.student_responses = {
            'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'questions': [],
            'total_time': 0,
            'total_score': 0
        }

    def generate_questions(self, text, num_questions=4, pool_size=10):
        theory_prompts = [
            f"Generate a detailed theoretical question about this concept: {text}",
            f"Ask a question that requires explaining a concept from this text: {text}",
            f"Create a question that tests deep understanding of: {text}",
            f"Form a question that requires detailed explanation of a topic from: {text}",
            f"Generate a question that explores the implications of: {text}",
            f"Ask a question that requires critical analysis of: {text}",
            f"Create a question that tests application of: {text}",
            f"Form a question that requires comparison of concepts from: {text}",
            f"Generate a question that evaluates the significance of: {text}",
            f"Ask a question that requires synthesis of ideas from: {text}"
        ]

        generated_questions = []

        # Generate a pool of questions
        for prompt in theory_prompts[:pool_size]:
            question = self.question_generator(prompt, max_length=100, num_return_sequences=1)[0]['generated_text']
            answer = self.question_generator(f"Provide a detailed answer for: {question}\n\nBased on text: {text}",
                                          max_length=200, num_return_sequences=1)[0]['generated_text']

            generated_questions.append({
                'question': question,
                'answer': answer
            })

        # Randomly select 4 distinct questions
        selected_questions = self.select_distinct_questions(generated_questions, num_questions)
        self.qa_pairs = selected_questions

    def select_distinct_questions(self, questions, num_questions, threshold=0.7):
        selected_questions = []
        remaining_questions = questions.copy()

        while len(selected_questions) < num_questions and remaining_questions:
            # Randomly pick a question
            question = random.choice(remaining_questions)
            remaining_questions.remove(question)

            # Check if the question is distinct from already selected questions
            is_distinct = True
            for selected_question in selected_questions:
                similarity = float(util.pytorch_cos_sim(
                    self.similarity_model.encode(question['question'], convert_to_tensor=True),
                    self.similarity_model.encode(selected_question['question'], convert_to_tensor=True)
                )[0][0])
                if similarity > threshold:
                    is_distinct = False
                    break

            if is_distinct:
                selected_questions.append(question)

        return selected_questions

    def check_answer(self, user_answer, correct_answer):
        similarity = float(util.pytorch_cos_sim(
            self.similarity_model.encode(user_answer, convert_to_tensor=True),
            self.similarity_model.encode(correct_answer, convert_to_tensor=True)
        )[0][0])
        return similarity

    def calculate_marks(self, similarity):
        return round(similarity * 5, 2)  # Max 5 marks based on similarity

    def save_session_data(self, filename='student_responses.json'):
        with open(filename, 'w') as f:
            json.dump(self.student_responses, f, indent=2)

def run_quiz():
    #text = process_file()
    qs = QuestionSystem()
    qs.generate_questions(text)

    total_time = 0
    total_score = 0

    for i, qa in enumerate(qs.qa_pairs, 1):
        print(f"\nQuestion {i}: {qa['question']}")

        start_time = time.time()
        user_answer = input("Your answer: ")
        time_taken = round(time.time() - start_time, 2)

        similarity = qs.check_answer(user_answer, qa['answer'])
        marks = qs.calculate_marks(similarity)

        response_data = {
            'question': qa['question'],
            'correct_answer': qa['answer'],
            'user_answer': user_answer,
            'time_taken': time_taken,
            'similarity': round(similarity * 100, 2),
            'marks': marks
        }

        qs.student_responses['questions'].append(response_data)
        total_time += time_taken
        total_score += marks

        print(f"Time taken: {time_taken} seconds")
        print(f"Similarity: {similarity * 100:.2f}%")
        print(f"Marks: {marks}/5")

    qs.student_responses['total_time'] = round(total_time, 2)
    qs.student_responses['total_score'] = round(total_score, 2)

    print(f"\nQuiz Summary:")
    print(f"Total time: {total_time:.2f} seconds")
    print(f"Total score: {total_score:.2f}/20")
    print(f"Average time per question: {total_time/4:.2f} seconds")

    if round(total_score) < 8:
        print(f"Result: Fail")
    else:
        print(f"Result: Pass")

    qs.save_session_data()


In [13]:
# Run the system
run_quiz()

Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (643 > 512). Running this sequence through the model will result in indexing errors



Question 1: What is the difference between a frame by frame and a motion capture?
Your answer: idk
Time taken: 35.68 seconds
Similarity: 3.11%
Marks: 0.16/5

Question 2: What is the main purpose of this module?
Your answer: idk
Time taken: 7.72 seconds
Similarity: 5.89%
Marks: 0.29/5

Question 3: Which of the following is the best way to remove hidden surface problem?
Your answer: Visible surface detection or hidden surface removal
Time taken: 50.43 seconds
Similarity: 44.53%
Marks: 2.23/5

Question 4: What is the main purpose of a keyframe?
Your answer: To define the starting and ending points of an object or character's movement or pose
Time taken: 35.18 seconds
Similarity: 100.00%
Marks: 5.0/5

Quiz Summary:
Total time: 129.01 seconds
Total score: 7.68/20
Average time per question: 32.25 seconds
Result: Pass
